In [1]:
#0 import some functions
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from numpy import median,mean, std, histogram, arange, exp, zeros, ones, array, unique
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy.ndimage.filters import gaussian_filter, uniform_filter1d
from scipy.ndimage.measurements import label
import numpy as np
from astropy import wcs
from scipy.special import erf
from math import sqrt, log, sin, cos, asin, atan2, degrees, radians, pi
from astropy.table import Table, Column
import pandas as pd

In [2]:
for j in range(1,10):
    print "busy with ppmos=",j
    dir1 = "/Volumes/THEMBA/Real/ppmos"+str(j)+"/"
    dir3 = dir1+"linewidths/"
    tafile = open(dir3+"pp"+str(j)+"_table","w")
    tafile.write("N0. Source Name      Source ID                 l         b          V_cen\n")
    f = open(dir1+"original_files","r")
    
    cube_names = []
    gim = 1
    for line in f:
        x0   = str(line.split()[0])
        x_name    = str(x0)+'.fits'
        header  = fits.getheader(dir1+x_name)
        cube = fits.getdata(dir1+x_name)
        
        rmsSTD = std(cube)
        #2.4 Fit of negative part of the noise distribution
        def GaussianNoise(F,N0,s0):
            return N0*exp(-F**2/2/s0**2)        
        
        def rmsFit(Cube):
            ## Make histogram of voxel values in the cube
            binEdges      = arange(Cube.min(),abs(cube.min())/1000,abs(cube.min())/1000)
            bins          = (binEdges[1:]+binEdges[:-1])/2
            hist,binEdges = histogram(Cube, bins=binEdges)
            ## Fit histogram
            p0=[hist.max(),-bins[hist<hist.max()/2].max()*2/2.355]
            return curve_fit(GaussianNoise,hist,bins,p0=p0)[0][1]
        
    
        
        #3 Detection of voxels that contain signal
        #3.1 define kernel sizes for spatial 
        # and spectral smoothing & mask (initially empty)
        kernelsSpat = [0,3,6]
        kernelsSpec  = [0,3,7,15]
        mask = zeros(cube.shape)
        
        for kSpat in kernelsSpat:
            for kSpec in kernelsSpec:
                #print 'smoothing with kernel of size: [',kSpat,',',kSpat,',',kSpec,']'
                ### spatial smoothing
                smoothedCube = gaussian_filter(cube,[0,kSpat/2.355,kSpat/2.355],mode='constant')
                ### spectral smoothing
                if kSpec!=0:
                    smoothedCube = uniform_filter1d(smoothedCube,kSpec,axis=0,mode='constant')
                ### estimate noise for the smoothed cube
                rms = rmsFit(smoothedCube)
                ### add pixels above 3 sigma limit to mask
                #mask[smoothedCube>2.8*rms] = 1
                mask[smoothedCube>0] = 1 #raw image
                
        #4 Group identified voxels into sources
        maskIDs = label(mask)[0]
        
        #5 remove smaller detections and relabel sources
        counter = 1
        for ID in unique(maskIDs)[1:]:
            #remove all "islands" with #voxels<500
            if (maskIDs[maskIDs==ID].sum())/ID<1700:
                maskIDs[maskIDs==ID] = 0
            else:
                # assign a new ID to remaining sources
                maskIDs[maskIDs==ID] = counter
                counter+=1
        #fits.writeto(dir+'themba_msk.fits',maskIDs,clobber=True) #masked

        labeled_array, num_features = label(maskIDs)
        nf=num_features
        
        data = cube     #cube data
        mask = maskIDs  #masked cube
        nz, ny, nx = data.shape  # read cube dimensions
        Centroid = []
        Sum_flux = []
        for i in range(1,nf+1):
            centroid = [0.0, 0.0, 0.0]
            sum_flux = 0.0
            for x in range(nx):
                for y in range(ny):
                    for z in range(nz):
                        if mask[z][y][x] == i:
                            flux = data[z][y][x]
                            centroid[0] += flux * float(x)
                            centroid[1] += flux * float(y)
                            centroid[2] += flux * float(z)
                            sum_flux += flux

            centroid[0] /= sum_flux
            centroid[1] /= sum_flux
            centroid[2] /= sum_flux
            Centroid.append(centroid)
            Sum_flux.append(sum_flux)
            #print "for source #"+str(i)+", Flux Centroid : " + str(centroid[0]) + ", " + str(centroid[1]) + ", " + str(centroid[2])+". Flux sum ="+str(sum_flux)      
            
        # Convert position to WCS coordinates
        RA,Dec,Vel = [],[],[]
        wcshead = wcs.WCS(header)
        for i in range(nf):
            coordinates = wcshead.wcs_pix2world(Centroid[i][0], Centroid[i][1], Centroid[i][2], 0)
            ra = coordinates[0]
            dec = coordinates[1]
            vel = coordinates[2]*1e-3

            RA.append(ra)
            Dec.append(dec)
            Vel.append(vel)

            #print '--------------------------------------------------------------------------------------'
            #print "coord of source #1, "+str((RA[0],Dec[0],Vel[0]))
    

        # TASK 3
        # Convert from J2000 to Galactic coordinates
        Glat,Glon = [], []
        for i in range(nf):
            ra0 = radians(192.8595)
            dec0 = radians(27.1284)
            gl0 = radians(122.9320)
            glat = asin(sin(radians(Dec[i])) * sin(dec0) + cos(radians(Dec[i])) * cos(dec0) * cos(radians(RA[i]) - ra0))
            glon = gl0 - atan2(cos(radians(Dec[i])) * sin(radians(RA[i]) - ra0), sin(radians(Dec[i])) * cos(dec0) - cos(radians(Dec[i])) * sin(dec0) * cos(radians(RA[i]) - ra0))
            Glat.append(degrees(glat))
            Glon.append(degrees(glon))
        
        # Write functions that will convert RA [deg] >> RA [hm] & Dec [deg] >> Dec [dm]
        # Note you can change the ticks on the plots {from degrees to h:m & d:m} by replacing C1 & C2 by c1 & c2 respectively
        def hms(f):
            H = f/15.
            h = int(H)
            m = int((H-h)*60)
            s =round((((H-h)*60 - m)*60),2)
            return h,m,s
    
        def dms(f):
            d = int(f)
            m = int((f - d)*60.)
            s = round(((f - d)*60. - m)*60,1)
            return d, m,s
        # Apply above convertion
        RA_hms,Dec_dms = [],[]
        print "source #",gim,"and nf=",nf
        for i in range(nf):
            RA_hms.append(str(hms(RA[i])[0])+':'+str(hms(RA[i])[1])+':'+str(hms(RA[i])[2]))
            Dec_dms.append(str(dms(Dec[i])[0])+':'+str(dms(Dec[i])[1])+':'+str(dms(Dec[i])[2]))
        
        tafile.write(str(gim)+"     "+x0+"    "+RA_hms[0]+ "+"+Dec_dms[0]+ "    "
               +str(round(degrees(glon),3))+ "    " + str(round(degrees(glat),3))+
            "   "+ str(int(round(vel)))+"\n")
        gim = gim +1
    tafile.close()
    
print "FINISHED"       

busy with ppmos= 1


/Users/themba/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


source # 1 and nf= 1
source # 2 and nf= 1
FINISHED


Source Name    Source ID   RA    Dec   l    b    V_cen


file